<a href="https://colab.research.google.com/github/14Pranav/Major-Project/blob/main/Sentiments_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 5.6MB/s 


In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs=SentimentIntensityAnalyzer()
text='The product is really good'
vs.polarity_scores(text)

{'compound': 0.4927, 'neg': 0.0, 'neu': 0.556, 'pos': 0.444}

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [6]:
urls=['https://inshorts.com/en/read/sports',
      'https://inshorts.com/en/read/world',
      'https://inshorts.com/en/read/technology',
      'https://inshorts.com/en/read/national']
def build_dataset(urls):
  news_data=[]
  for url in urls:
    news_category=url.split('/')[-1]
    data=requests.get(url)
    soup=BeautifulSoup(data.content)
    news_articles=[{'news_headline':headline.find('span',attrs={"itemprop":"headline"}).string,
                    'news_article':article.find('div',attrs={"itemprop":"articleBody"}).string,
                    'news_category':news_category}
                    for headline,article in zip(soup.find_all('div',class_=["news-card-title news-right-box"]),
                     soup.find_all('div',class_=["news-card-content news-right-box"]))]
    news_articles=news_articles[0:20]
    news_data.extend(news_articles)

  df=pd.DataFrame(news_data)
  df=df[['news_headline','news_article','news_category']]
  return df

In [7]:
df=build_dataset(urls)
df.head()

,news_headline,news_article,news_category
0,Picture of Dwayne Bravo 'stealing' a run goes ...,A picture of CSK's Dwayne Bravo being way out ...,sports
1,Shubman's family receives Mahindra Thar gifted...,Team India batsman Shubman Gill took to social...,sports
2,Beckham responds to European Super League plan...,After 12 of Europe's leading football clubs in...,sports
3,RR get in ICU once Buttler and Samson are dism...,Ex-India batsman Aakash Chopra said Rajasthan ...,sports
4,Challenge would be to switch on mentally on Ch...,Ahead of playing Mumbai Indians at MA Chidamba...,sports


In [8]:
df.to_csv('news.csv',index=False)


In [9]:
df=pd.read_csv('/content/news.csv')
df

,news_headline,news_article,news_category
0,Picture of Dwayne Bravo 'stealing' a run goes ...,A picture of CSK's Dwayne Bravo being way out ...,sports
1,Shubman's family receives Mahindra Thar gifted...,Team India batsman Shubman Gill took to social...,sports
2,Beckham responds to European Super League plan...,After 12 of Europe's leading football clubs in...,sports
3,RR get in ICU once Buttler and Samson are dism...,Ex-India batsman Aakash Chopra said Rajasthan ...,sports
4,Challenge would be to switch on mentally on Ch...,Ahead of playing Mumbai Indians at MA Chidamba...,sports
...,...,...,...
75,Govt restores 20 control rooms to address migr...,The Ministry of Labour and Employment has reju...,national
76,Impose 'weekend janata curfews' in Karnataka: ...,The COVID-19 Technical Advisory Committee (TAC...,national
77,COVID-19 2nd wave brought a lot of challenges:...,Maharashtra Health Minister Rajesh Tope on Tue...,national
78,Kerala's Kasargod withdraws 'COVID-19 negative...,The Kasargod District Collector has withdrawn ...,national


In [10]:
import nltk
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove("no")
stopword_list.remove("not")
len(stopword_list)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


177

In [11]:
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text

In [12]:
!pip install contractions

     |████████████████████████████████| 327kB 5.2MB/s 
     |████████████████████████████████| 286kB 7.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85396 sha256=ac38a40cf9bd20df84aecf9af6fd0b0ff43daaec92304b887358a5443e3a67e7
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [13]:
import contractions
def con(text):
  expand=contractions.fix(text)
  return expand

In [14]:
import re
def remove_sp(text):
  pattern=r'[^A-Za-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text

In [15]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

In [16]:
def remove_stopwords(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip()for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text

In [17]:
df.news_headline=df.news_headline.apply(lambda x:x.lower())
df.news_article=df.news_article.apply(lambda x:x.lower())

df.news_headline=df.news_headline.apply(html_tag)
df.news_article=df.news_article.apply(html_tag)

df.news_headline=df.news_headline.apply(con)
df.news_article=df.news_article.apply(con)

df.news_headline=df.news_headline.apply(remove_sp)
df.news_article=df.news_article.apply(remove_sp)

df.news_headline=df.news_headline.apply(remove_stopwords)
df.news_article=df.news_article.apply(remove_stopwords)

df.head()

,news_headline,news_article,news_category
0,picture dwayne bravo stealing run goes viral p...,picture csks dwayne bravo way crease nonstrike...,sports
1,shubmans family receives mahindra thar gifted ...,team india batsman shubman gill took social me...,sports
2,beckham responds european super league plan sa...,12 europes leading football clubs including ma...,sports
3,rr get icu buttler samson dismissed aakash chopra,exindia batsman aakash chopra said rajasthan r...,sports
4,challenge would switch mentally chennai wicket...,ahead playing mumbai indians chidambaram stadi...,sports


In [18]:
df['compound']=df['news_article'].apply(lambda x:vs.polarity_scores(x)['compound'])

In [19]:
df

,news_headline,news_article,news_category,compound
0,picture dwayne bravo stealing run goes viral p...,picture csks dwayne bravo way crease nonstrike...,sports,-0.1823
1,shubmans family receives mahindra thar gifted ...,team india batsman shubman gill took social me...,sports,0.8555
2,beckham responds european super league plan sa...,12 europes leading football clubs including ma...,sports,0.9584
3,rr get icu buttler samson dismissed aakash chopra,exindia batsman aakash chopra said rajasthan r...,sports,-0.0516
4,challenge would switch mentally chennai wicket...,ahead playing mumbai indians chidambaram stadi...,sports,0.7096
...,...,...,...,...
75,govt restores 20 control rooms address migrant...,ministry labour employment rejuvenated 20 cont...,national,-0.6369
76,impose weekend janata curfews karnataka expert...,covid19 technical advisory committee tac const...,national,0.1027
77,covid19 2nd wave brought lot challenges mahara...,maharashtra health minister rajesh tope tuesda...,national,0.0516
78,keralas kasargod withdraws covid19 negative re...,kasargod district collector withdrawn order ma...,national,-0.5267
